In [3]:
%load_ext dockermagic

# Hadoop - multi-node cluster setup 
![Hadoop](https://hadoop.apache.org/elephant.png)

## Create Hadoop base image

### Create docker container

- Ubuntu 20.04 (https://ubuntu.com/)
- Docker (https://www.docker.com/)
    - container based virtualization

In [7]:
%%bash

docker run -d -t --rm --name hadoopimg -h hadoopimg ubuntu:20.04

docker ps

a9f89cb975f3e4bce7a3c84518d0b80f844c4474c237947b50b8c85d63beddc3


CONTAINER ID   IMAGE          COMMAND       CREATED        STATUS                  PORTS     NAMES
a9f89cb975f3   ubuntu:20.04   "/bin/bash"   1 second ago   Up Less than a second             hadoopimg


### Install Dependencies

- Java 8 (OpenJDK) - https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions
- Other packages: ssh pdsh wget apt-utils

In [8]:
%%dockerexec hadoopimg

# Update package list
apt -qq update > /install.log 2>&1

# Install and configure tzdata
DEBIAN_FRONTEND=noninteractive apt -qq install -y tzdata >> /install.log 2>&1
ln -fs /usr/share/zoneinfo/America/Sao_Paulo /etc/localtime >> /install.log 2>&1
dpkg-reconfigure --frontend noninteractive tzdata >> /install.log 2>&1

# Install Hadoop dependencies
apt -qq -f -y install openjdk-8-jdk ssh pdsh >> /install.log 2>&1

# Install other dependencies
apt -qq -f -y install vim wget apt-utils python3 python3-pip \
    ipython3 less unzip sudo net-tools >> /install.log 2>&1

### Install Hadoop

- http://hadoop.apache.org/
- Version 3.3.6
- Base directory: /opt
- User/group: hadoop/hadoop
- Package with binaries (version 3.3.6): https://hadoop.apache.org/releases.html

In [9]:
%%dockerexec hadoopimg

# Enable rwx for all on /opt
chmod 777 /opt

# Create user/group hadoop
useradd -m -U -s /bin/bash hadoop

# Enable sudo for hadoop
sed -i "\$ahadoop  ALL=(ALL) NOPASSWD:ALL" /etc/sudoers

In [10]:
%%bash

HADOOPVERSION=hadoop-3.3.6

# Download package
cd ../pkgs
wget -q -c https://dlcdn.apache.org/hadoop/common/$HADOOPVERSION/$HADOOPVERSION.tar.gz

# Copy installation package to container
docker cp $HADOOPVERSION.tar.gz hadoopimg:/opt

In [11]:
%%dockerexec -u hadoop hadoopimg

HADOOPVERSION=hadoop-3.3.6

# Modify user/group permissions and unpack file
sudo chown hadoop:hadoop /opt/$HADOOPVERSION.tar.gz
tar -zxf /opt/$HADOOPVERSION.tar.gz -C /opt
rm /opt/$HADOOPVERSION.tar.gz

# Create link
ln -s /opt/$HADOOPVERSION /opt/hadoop

### Configure environment variables

- Create file /opt/envvars.sh with environment variables

In [12]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/envvars.sh << EOF
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_VERSION=3.3.6
export HADOOP_COMMON_HOME=\${HADOOP_HOME}
export HADOOP_CONF_DIR=\${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=\${HADOOP_HOME}
export HADOOP_MAPRED_HOME=\${HADOOP_HOME}
export HADOOP_YARN_HOME=\${HADOOP_HOME}

export PATH=\${PATH}:\${HADOOP_HOME}/bin:\${HADOOP_HOME}/sbin     

EOF

cat /opt/envvars.sh

export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
export PDSH_RCMD_TYPE=ssh

export HADOOP_HOME=/opt/hadoop
export HADOOP_COMMON_HOME=${HADOOP_HOME}
export HADOOP_CONF_DIR=${HADOOP_HOME}/etc/hadoop
export HADOOP_HDFS_HOME=${HADOOP_HOME}
export HADOOP_MAPRED_HOME=${HADOOP_HOME}
export HADOOP_YARN_HOME=${HADOOP_HOME}

export PATH=${PATH}:${HADOOP_HOME}/bin:${HADOOP_HOME}/sbin     



### Configure passwordless ssh

In [13]:
%%dockerexec -u hadoop hadoopimg

# Disable host key checking
sudo tee -a /etc/ssh/ssh_config << EOF
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
EOF

# Create ssh key
ssh-keygen -q -t rsa -P "" -f ~/.ssh/id_rsa

# Copy public key to authorized_keys file
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys

    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null


### Hadoop configuration files

- Hadoop configuration files location: \$HADOOP\_HOME\/etc\/hadoop
- All cluster nodes contain the same files

#### hadoop-env.sh

- Definition of environment variables used by Hadoop processes

In [14]:
%%dockerexec -u hadoop hadoopimg

cat >> /opt/hadoop/etc/hadoop/hadoop-env.sh << EOF
export JAVA_HOME=/usr/lib/jvm/java-1.8.0-openjdk-amd64
EOF

#### core-site.xml

- Hadoop main configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-common/core-default.xml

In [15]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/core-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>fs.defaultFS</name>
    <value>hdfs://hadoop:9000</value>
</property>

<property>
    <name>hadoop.proxyuser.hadoop.groups</name>
    <value>*</value>
</property>

<property>
    <name>hadoop.proxyuser.hadoop.hosts</name>
    <value>*</value>
</property>

</configuration>
EOF

#### hdfs-site.xml

- HDFS configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml

In [16]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/hdfs-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>dfs.namenode.name.dir</name>
    <value>/opt/hadoop/data/nameNode</value>
</property>

<property>
    <name>dfs.datanode.data.dir</name>
    <value>/opt/hadoop/data/dataNode</value>
</property>

<property>
    <name>dfs.replication</name>
    <value>2</value>
</property>

<property>
    <name>dfs.blocksize</name>
    <value>33554432</value>
</property>

<property>
    <name>dfs.hosts.exclude</name>
    <value>/opt/hadoop/etc/hadoop/dfs.exclude</value>
</property>

<property>
    <name>dfs.namenode.heartbeat.recheck-interval</name>
    <value>10000</value>
</property>

</configuration>

EOF

#### yarn-site.xml

- YARN configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-yarn/hadoop-yarn-common/yarn-default.xml

In [17]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/yarn-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<configuration>

<property>
    <name>yarn.resourcemanager.hostname</name>
    <value>hadoop</value>
</property>

<property>
    <name>yarn.nodemanager.aux-services</name>
    <value>mapreduce_shuffle</value>
</property>

<property>
    <name>yarn.nodemanager.resource.memory-mb</name>
    <value>1536</value>
</property>

<property>
    <name>yarn.scheduler.maximum-allocation-mb</name>
    <value>1536</value>
</property>

<property>
    <name>yarn.scheduler.minimum-allocation-mb</name>
    <value>128</value>
</property>

<property>
    <name>yarn.timeline-service.enabled</name>
    <value>true</value>
</property>

<property>
    <name>yarn.timeline-service.hostname</name>
    <value>hadoop</value>
</property>

<property>
    <name>yarn.system-metrics-publisher.enabled</name>
    <value>true</value>
</property>

<property>
    <name>yarn.log-aggregation-enable</name>
    <value>true</value>
</property>

<property>
    <name>yarn.nm.liveness-monitor.expiry-interval-ms</name>
    <value>10000</value>
</property>

</configuration>
EOF

#### mapred-site.xml

- MapReduce configuration
- Default parameters: http://hadoop.apache.org/docs/r3.2.1/hadoop-mapreduce-client/hadoop-mapreduce-client-core/mapred-default.xml

In [18]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/mapred-site.xml << EOF
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<configuration>

<property>
    <name>mapreduce.framework.name</name>
    <value>yarn</value>
</property>

<property>
    <name>mapreduce.application.classpath</name>
    <value>/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/mapreduce/lib/*</value>
</property>

<property>
    <name>yarn.app.mapreduce.am.resource.mb</name>
    <value>512</value>
</property>

<property>
    <name>mapreduce.map.memory.mb</name>
    <value>256</value>
</property>

<property>
    <name>mapreduce.reduce.memory.mb</name>
    <value>256</value>
</property>

</configuration>
EOF

#### workers

- List of worker nodes (NodeManager and DataNode)

In [19]:
%%dockerexec -u hadoop hadoopimg

cat > /opt/hadoop/etc/hadoop/workers << EOF
hadoop1
hadoop2
hadoop3
EOF

### Commit base image

In [20]:
%%bash

# Create hadoopimg image based on hadoop container
docker commit hadoopimg hadoopimg

# Stop base container
docker stop hadoopimg

sha256:19906e49e3fe6af61990eed834d7aabd462aac06a97cb33d442486a4c7914689
hadoopimg


## Create cluster

### Run nodes

In [21]:
%%bash

# MASTER

# Ports
# 9870 - Namenode
# 9868 - Secondary Namenode
# 8088 - ResourceManager
# 19888 - MapReduce Job History
# 8188 - Timeline Service
# 4040 - Spark Application UI
# 8080 - Jupyter

cd ..
docker run -d -t --memory 4g --memory-swap 4g --rm --name hadoop -h hadoop -u hadoop \
    -v "$(pwd)"/pkgs:/opt/pkgs -v "$(pwd)"/notebooks:/opt/notebooks -v "$(pwd)"/datasets:/opt/datasets \
    -p 9870:9870 -p 9868:9868 -p 8088:8088 -p 19888:19888 -p 8188:8188 -p 4040:4040 -p 8080:8080 hadoopimg

# WORKERS

# Ports
# 9864 - DataNode WebUI
# 8042 - NodeManager WebUI

# Hadoop1
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop1 -h hadoop1 -u hadoop \
    -p 9864:9864 -p 8042:8042 hadoopimg
# Hadoop2
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop2 -h hadoop2 -u hadoop \
    -p 9865:9864 -p 8043:8042  hadoopimg
# Hadoop3
docker run -d -t --memory 2g --memory-swap 2g --rm --name hadoop3 -h hadoop3 -u hadoop \
    -p 9866:9864 -p 8044:8042  hadoopimg

docker ps

eb32245ae52436122205fa3247cff44ae7b4c6789ef5ae83823a8e155b2a74ca


6e5189cc717145789a4b394c20541bc60fc410665127902ae66abbeccc70c7c1
50a9def6bb153fe3668d800ea70f2ece39126f33c07505f439b47a1df7e39acd
a59f1994b51c91e0eb87ca950e3a8a3ab7c38efee278be0f34c7801798b1d165
CONTAINER ID   IMAGE       COMMAND       CREATED                  STATUS                  PORTS                                                                                                                                                                                                                                                                                                             NAMES
a59f1994b51c   hadoopimg   "/bin/bash"   Less than a second ago   Up Less than a second   0.0.0.0:8044->8042/tcp, :::8044->8042/tcp, 0.0.0.0:9866->9864/tcp, :::9866->9864/tcp                                                                                                                                                                                                                              hadoop3

### Configure hosts file on all nodes

- /etc/hosts

In [22]:
%%bash

# Get IPs
M=$(docker inspect hadoop | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H1=$(docker inspect hadoop1 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H2=$(docker inspect hadoop2 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)
H3=$(docker inspect hadoop3 | grep \"IPAddress\" | head -1 | awk '{ print $2 }' | tr -d \",)

# Create hosts file
cat > hosts << EOF  
$M hadoop
$H1 hadoop1
$H2 hadoop2
$H3 hadoop3
EOF

cat hosts

# Copy to all nodes
docker exec -i -u root hadoop sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop1 sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop2 sh -c 'cat >> /etc/hosts' < hosts
docker exec -i -u root hadoop3 sh -c 'cat >> /etc/hosts' < hosts

# Remove local file
rm hosts

172.17.0.2 hadoop
172.17.0.3 hadoop1
172.17.0.4 hadoop2
172.17.0.5 hadoop3


### Start ssh server on all nodes

In [23]:
%%bash

for HOST in hadoop hadoop1 hadoop2 hadoop3
do
    echo $HOST
    docker exec -u root $HOST service ssh restart
    docker exec -u root $HOST service ssh status
done

hadoop
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop1
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop2
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running
hadoop3
 * Restarting OpenBSD Secure Shell server sshd
   ...done.
 * sshd is running


## Format HDFS on Namenode

In [24]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs namenode -format -force -nonInteractive

2023-11-23 08:38:39,686 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = hadoop/172.17.0.2
STARTUP_MSG:   args = [-format, -force, -nonInteractive]
STARTUP_MSG:   version = 3.3.6
STARTUP_MSG:   classpath = /opt/hadoop/etc/hadoop:/opt/hadoop/share/hadoop/common/lib/avro-1.7.7.jar:/opt/hadoop/share/hadoop/common/lib/kerby-config-1.0.1.jar:/opt/hadoop/share/hadoop/common/lib/httpclient-4.5.13.jar:/opt/hadoop/share/hadoop/common/lib/reload4j-1.2.22.jar:/opt/hadoop/share/hadoop/common/lib/kerby-asn1-1.0.1.jar:/opt/hadoop/share/hadoop/common/lib/kerby-pkix-1.0.1.jar:/opt/hadoop/share/hadoop/common/lib/curator-recipes-5.2.0.jar:/opt/hadoop/share/hadoop/common/lib/kerby-xdr-1.0.1.jar:/opt/hadoop/share/hadoop/common/lib/token-provider-1.0.1.jar:/opt/hadoop/share/hadoop/common/lib/hadoop-annotations-3.3.6.jar:/opt/hadoop/share/hadoop/common/lib/listenablefuture-9999.0-empty-to-avoid-conflict-w

## Start Hadoop daemons

- manual execution: ```hdfs --daemon start (namenode|datanode)``` and ```yarn --daemon start (resourcemanager|nodemanager)```
- auxilliary scripts to run all processes on the cluster: start-dfs.sh (HDFS) and start-yarn.sh (YARN)
- some services still need to be executed manually (timelineserver, historyserver)

In [25]:
%%dockerexec hadoop

source /opt/envvars.sh

# HDFS
start-dfs.sh

# YARN
start-yarn.sh

# timelineserver
yarn --daemon start timelineserver

# historyserver
mapred --daemon start historyserver

Starting namenodes on [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
Starting datanodes
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
hadoop2: WARNING: /opt/hadoop-3.3.6/logs does not exist. Creating.
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hosts.
hadoop3: WARNING: /opt/hadoop-3.3.6/logs does not exist. Creating.
hadoop1: WARNING: /opt/hadoop-3.3.6/logs does not exist. Creating.
Starting secondary namenodes [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
Starting resourcemanager
Starting nodemanagers
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop1: Warning:

In [26]:
%%bash

# Listing all processes
for HOST in hadoop hadoop1 hadoop2 hadoop3; do
    echo $HOST
    docker exec $HOST jps
done

hadoop
721 ApplicationHistoryServer
225 NameNode
418 SecondaryNameNode
595 ResourceManager
1028 Jps
781 JobHistoryServer
hadoop1
197 NodeManager
294 Jps
92 DataNode
hadoop2
293 Jps
197 NodeManager
92 DataNode
hadoop3
196 NodeManager
293 Jps
91 DataNode


## Create HDFS directories

In [27]:
%%dockerexec hadoop

source /opt/envvars.sh

hdfs dfs -mkdir -p /user/hadoop
hdfs dfs -chown hadoop:hadoop /user/hadoop
hdfs dfs -mkdir /tmp
hdfs dfs -chmod 777 /tmp

mkdir: `/tmp': File exists


## Install and run Jupyter on master node

In [29]:
%%dockerexec hadoop

# pip3 -q install notebook
pip3 -q install pip
pip3 -q install traitlets==5.6.0
pip3 -q install jupyterlab

IP=$(ifconfig eth0 | grep inet | awk '{ print $2 }')

cd /opt

export SHELL=/bin/bash
# nohup /home/hadoop/.local/bin/jupyter-notebook --ip=$IP --port=8080 --NotebookApp.token='' --NotebookApp.password='' --NotebookApp.notebook_dir='/' --no-browser &
nohup /home/hadoop/.local/bin/jupyter-lab --ip=$IP --port=8080 --notebook-dir='/' --ServerApp.token='' --ServerApp.password='' --no-browser &

echo $! > /tmp/jupyter.pid

# To kill
# kill $(cat /tmp/jupyter.pid)

[W 2023-11-23 08:50:35.211 ServerApp] ServerApp.token config is deprecated in 2.0. Use IdentityProvider.token.
[I 2023-11-23 08:50:35.222 ServerApp] Package jupyterlab took 0.0000s to import
[I 2023-11-23 08:50:35.231 ServerApp] Package jupyter_lsp took 0.0083s to import
[W 2023-11-23 08:50:35.231 ServerApp] A `_jupyter_server_extension_points` function was not found in jupyter_lsp. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyter Server.
[I 2023-11-23 08:50:35.235 ServerApp] Package jupyter_server_terminals took 0.0034s to import
[I 2023-11-23 08:50:35.236 ServerApp] Package notebook_shim took 0.0000s to import
[W 2023-11-23 08:50:35.236 ServerApp] A `_jupyter_server_extension_points` function was not found in notebook_shim. Instead, a `_jupyter_server_extension_paths` function was found and will be used for now. This function name will be deprecated in future releases of Jupyt

## Access web interfaces

- Jupyterlab
    - http://localhost:8080
- Master - hadoop
    - Resource Manager: http://localhost:8088
    - NameNode: http://localhost:9870
    - Secondary NameNode: http://localhost:9868
    - MapReduce Job History: http://localhost:19888
    - Timeline Service: http://localhost:8188
- Workers
    - hadoop1
        - NodeManager: http://localhost:8042
        - DataNode: http://localhost:9864
    - hadoop2
        - NodeManager: http://localhost:8043
        - DataNode: http://localhost:9865
    - hadoop3
        - NodeManager: http://localhost:8044
        - DataNode: http://localhost:9866

## Run mapreduce Pi example

In [32]:
%%dockerexec hadoop

HADOOP_VERSION=3.3.6

source /opt/envvars.sh
cd /opt/hadoop/share/hadoop/mapreduce

hadoop jar ./hadoop-mapreduce-examples-$HADOOP_VERSION.jar pi 6 10000

Number of Maps  = 6
Samples per Map = 10000
Wrote input for Map #0
Wrote input for Map #1
Wrote input for Map #2
Wrote input for Map #3
Wrote input for Map #4
Wrote input for Map #5
Starting Job
2023-11-23 08:56:20,836 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at hadoop/172.17.0.2:8032
2023-11-23 08:56:21,154 INFO client.AHSProxy: Connecting to Application History server at hadoop/172.17.0.2:10200
2023-11-23 08:56:21,451 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/hadoop/.staging/job_1700739592673_0001
2023-11-23 08:56:22,224 INFO input.FileInputFormat: Total input files to process : 6
2023-11-23 08:56:22,331 INFO mapreduce.JobSubmitter: number of splits:6
2023-11-23 08:56:22,518 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1700739592673_0001
2023-11-23 08:56:22,518 INFO mapreduce.JobSubmitter: Executing with tokens: []
2023-11-23 08:56:22,712 INFO conf.Configuration: resource-types.x

# SHUTDOWN PROCEDURE

## Stop Jupyter

In [36]:
%%dockerexec hadoop

kill $(cat /tmp/jupyter.pid)

## Stop Hadoop daemons

In [37]:
%%dockerexec hadoop

source /opt/envvars.sh

stop-dfs.sh
stop-yarn.sh
yarn --daemon stop timelineserver
mapred --daemon stop historyserver

Stopping namenodes on [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
Stopping datanodes
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hosts.
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
Stopping secondary namenodes [hadoop]
hadoop: Warning: Permanently added 'hadoop,172.17.0.2' (ECDSA) to the list of known hosts.
2023-03-27 13:53:53,388 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping nodemanagers
hadoop3: Warning: Permanently added 'hadoop3,172.17.0.5' (ECDSA) to the list of known hosts.
hadoop2: Warning: Permanently added 'hadoop2,172.17.0.4' (ECDSA) to the list of known hosts.
hadoop1: Warning: Permanently added 'hadoop1,172.17.0.3' (ECDSA) to the list of known hos

## Stop Docker containers

In [38]:
%%bash

for HOST in hadoop hadoop1 hadoop2 hadoop3; do
    docker stop $HOST
done

docker ps

hadoop
hadoop1
hadoop2
hadoop3
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
